### Download SpeechBrain-MOABB

In [3]:
%%capture
%pip install --upgrade pip
!git clone https://github.com/speechbrain/benchmarks
%cd /home/aquan/dev/COMP432/speechbrain_moabb_project/benchmarks
!git checkout eeg

%pip install -r /home/aquan/dev/COMP432/speechbrain_moabb_project/benchmarks/benchmarks/MOABB/extra-requirements.txt # Install additional dependencies

### Install the requirements

In [4]:
%%capture
# Clone SpeechBrain repository (development branch)
%cd /home/aquan/dev/COMP432/speechbrain_moabb_project/
!git clone https://github.com/speechbrain/speechbrain/

# Install required dependencies
%pip install -r /home/aquan/dev/COMP432/speechbrain_moabb_project/speechbrain/requirements.txt

# Install SpeechBrain in editable mode
%pip install -e /home/aquan/dev/COMP432/speechbrain_moabb_project/speechbrain/.

%cd /home/aquan/dev/COMP432/speechbrain_moabb_project/

In [5]:
%pip install -q braindecode

import mne
import moabb
import orion
import sklearn
import torch
import torchinfo
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml, dump_hyperpyyaml
import braindecode

Note: you may need to restart the kernel to use updated packages.


<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/aquan/.local/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
/home/aquan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidge

### Getting model/hyperparams files

In [74]:
# get the path to the model files
eegnet_model_path = "/home/aquan/dev/COMP432/speechbrain_moabb_project/my_models/EEGNet/EEGNet.py"
eegnet_fusion_model_path = "/home/aquan/dev/COMP432/speechbrain_moabb_project/my_models/EEGNetFusion/EEGNetFusion.py"
deep_conv_net_model_path = "/home/aquan/dev/COMP432/speechbrain_moabb_project/my_models/DeepConvNet/DeepConvNet.py"

In [75]:
# get the path to the hyperparameter files
eegnet_hpars_path = "/home/aquan/dev/COMP432/speechbrain_moabb_project/my_models/EEGNet/EEGNet.yaml"
eegnet_fusion_hpars_path = "/home/aquan/dev/COMP432/speechbrain_moabb_project/my_models/EEGNetFusion/EEGNetFusion.yaml"
deep_conv_net_hpars_path = "/home/aquan/dev/COMP432/speechbrain_moabb_project/my_models/DeepConvNet/DeepConvNet.yaml"

In [82]:
# current model that im working with
my_model_path = eegnet_fusion_model_path
my_hyperparameters_path = eegnet_fusion_hpars_path
my_model = ""

In [153]:
# load the model
with open(my_model_path, "r") as file:
    my_model = file.read()

# save the model into the models folder
f = open('/home/aquan/dev/COMP432/speechbrain_moabb_project/benchmarks/benchmarks/MOABB/models/MyModel.py', "w")
f.write(my_model)
f.close()

### Train the neural network on a single cross-validation fold

In [152]:
%cd /home/aquan/dev/COMP432/speechbrain_moabb_project/benchmarks/benchmarks/MOABB/

!python train.py {my_hyperparameters_path} \
--data_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/data/BNCI2014001' \
--cached_data_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/data' \
--output_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/results/single-fold-example/BNCI2014001' \
--data_iterator_name 'leave-one-session-out' \
--target_subject_idx 0 \
--target_session_idx 1 \
--number_of_epochs 600 \
--device 'cpu' # Switch to cuda for a speed up.


/home/aquan/dev/COMP432/speechbrain_moabb_project/benchmarks/benchmarks/MOABB
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/aquan/.local/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
SpeechBrain could not find any working torchaudio backe

### Run a complete experiment by looping over the entire dataset



In [155]:
!./run_experiments.sh --hparams {my_hyperparameters_path} \
--data_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/data/BNCI2014001'\
--cached_data_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/data' \
--output_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/results/full-experiment/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--number_of_epochs 400 \
--device 'cpu'

hparams: /home/aquan/dev/COMP432/speechbrain_moabb_project/my_models/EEGNetFusion/EEGNetFusion.yaml
data_folder: /home/aquan/dev/COMP432/speechbrain_moabb_project/data/BNCI2014001
cached_data_folder: /home/aquan/dev/COMP432/speechbrain_moabb_project/data
output_folder: /home/aquan/dev/COMP432/speechbrain_moabb_project/results/full-experiment/BNCI2014001
nsbj: 9
nsess: 2
seed: 9909
nruns: 1
eval_metric: acc
eval_set: test
train_mode: leave-one-session-out
rnd_dir: False
additional flags: --number_of_epochs 400 --device cpu 
Subject 0


<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/aquan/.local/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubl

### Hyperparameter tuning

In [ ]:
%cd /home/aquan/dev/COMP432/speechbrain_moabb_project/benchmarks/benchmarks/MOABB/

!./run_hparam_optimization.sh --hparams {my_hyperparameters_path} \
--data_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/data/BNCI2014001'\
--cached_data_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/data' \
--output_folder '/home/aquan/dev/COMP432/speechbrain_moabb_project/results/hyperparameter-search/BNCI2014001' \
--nsbj 9 --nsess 2 --nruns 1 --train_mode 'leave-one-session-out' \
--exp_name 'hyperparameter-search' \
--nsbj_hpsearch 1 --nsess_hpsearch 1 \
--nruns_eval 1 \
--eval_metric acc \
--exp_max_trials 5